In [3]:
!pip install PyPDF2 nltk pandas

  Using cached pypdf2-3.0.1-py3-none-any.whl.metadata (6.8 kB)
  Using cached nltk-3.9.1-py3-none-any.whl.metadata (2.9 kB)
  Using cached click-8.2.1-py3-none-any.whl.metadata (2.5 kB)
  Using cached joblib-1.5.1-py3-none-any.whl.metadata (5.6 kB)
  Using cached regex-2024.11.6-cp313-cp313-macosx_11_0_arm64.whl.metadata (40 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
Using cached pypdf2-3.0.1-py3-none-any.whl (232 kB)
Using cached nltk-3.9.1-py3-none-any.whl (1.5 MB)
Using cached regex-2024.11.6-cp313-cp313-macosx_11_0_arm64.whl (284 kB)
Using cached click-8.2.1-py3-none-any.whl (102 kB)
Using cached joblib-1.5.1-py3-none-any.whl (307 kB)
Using cached tqdm-4.67.1-py3-none-any.whl (78 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6/6 [nltk]5/6 [nltk]b]


In [4]:
import re
import nltk
import pandas as pd
from PyPDF2 import PdfReader
from nltk.tokenize import sent_tokenize

nltk.download('punkt')


[nltk_data] Downloading package punkt to /Users/marsanto/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
# Helper functions
def extract_text_from_pdf(pdf_path):
    reader = PdfReader(pdf_path)
    text = ""
    for page in reader.pages:
        page_text = page.extract_text()
        if page_text:
            text += page_text + " "
    return text

def clean_text(text):
    text = re.sub(r"\s+", " ", text)
    return text.strip()

def extract_sentences(text):
    return sent_tokenize(text)

def is_high_risk(sentence):
    sentence_lower = sentence.lower()
    
    high_risk_triggers = [
        "loss of customer", "significant customer", "limited number of customers",
        "limited number of suppliers", "depend on", "rely on", "material adverse effect",
        "our largest customer", "one or more customers", "critical supplier", 
        "top customer", "major customer", "accounted for more than"
    ]
    
    return any(trigger in sentence_lower for trigger in high_risk_triggers)

def extract_type(sentence):
    sentence_lower = sentence.lower()
    if "supplier" in sentence_lower:
        return "supplier"
    elif "customer" in sentence_lower:
        return "customer"
    return ""

def extract_percentage(sentence):
    match = re.search(r'(\d{1,3}\.\d{1,2}%)', sentence)
    return match.group(1) if match else ""


In [ ]:
# Analyze the PDF
pdf_path = "/Users/marsanto/DevProjects/python/risk-extraction/data/Comfort Systems.pdf"  # using 1 pdf for testing
raw_text = extract_text_from_pdf(pdf_path)
cleaned_text = clean_text(raw_text)
sentences = extract_sentences(cleaned_text)

results = []

for sentence in sentences:
    if "customer" in sentence.lower() or "supplier" in sentence.lower():
        risk_flag = is_high_risk(sentence)
        if risk_flag:  # Only flag sentences considered risky
            results.append({
                "sentence": sentence.strip(),
                "percentage": extract_percentage(sentence),
                "type": extract_type(sentence),
                "is_high_risk": risk_flag
            })


LookupError: 
**********************************************************************
  Resource [93mpunkt_tab[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt_tab')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt_tab/english/[0m

  Searched in:
    - '/Users/marsanto/nltk_data'
    - '/Users/marsanto/DevProjects/python/hello-pandas/.venv/nltk_data'
    - '/Users/marsanto/DevProjects/python/hello-pandas/.venv/share/nltk_data'
    - '/Users/marsanto/DevProjects/python/hello-pandas/.venv/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
**********************************************************************


In [ ]:
# Export to CSV
df = pd.DataFrame(results)
df.to_csv("risk_sentences.csv", index=False)
df.head(10)